In [2]:
import requests
import pandas as pd
import math
import data_reader
import collections
import statistics
from tqdm import tqdm

In [3]:
stocktwit_df = data_reader.read_data4("lexicon", True)
stocktwit_df

,Sentiment,User_id,Message,Date,Time,Symbol
0,Bearish,442859953,crypto is even worthless if there is no world,2022-03-10,08:39:18Z,SPY
1,Bearish,442859939,why is it crashing so fast anyone know,2022-03-10,08:39:09Z,SPY
2,Bearish,442859938,bulls got trapped again how can it be l m f a o,2022-03-10,08:39:08Z,SPY
3,Bearish,442859889,by friday cmon make me rich daddy,2022-03-10,08:38:22Z,SPY
4,Bearish,442859884,algo last night predicts,2022-03-10,08:38:19Z,SPY
...,...,...,...,...,...,...
680928,Bullish,235142330,bullish rsi,2020-08-10,18:46:14Z,SOL.X
680929,Bullish,234896323,time to back up the truck and load up,2020-08-09,22:37:22Z,SOL.X
680930,Bullish,231400354,this things going to pump harder than arnold s...,2020-07-28,01:25:14Z,SOL.X
680931,Bullish,231399519,daddy wants gains,2020-07-28,01:19:47Z,SOL.X


In [4]:
# P(w) - total no. of occurence of w
corpus = stocktwit_df['Message'].str.cat(sep=' ')
corpus_list = corpus.split()
counter = collections.Counter(corpus_list)
word_occurence = dict(counter)
word_occurence

{'crypto': 26145,
 'is': 172039,
 'even': 12678,
 'worthless': 684,
 'if': 41969,
 'there': 15661,
 'no': 25487,
 'world': 8206,
 'why': 12106,
 'it': 94175,
 'crashing': 1117,
 'so': 34162,
 'fast': 2855,
 'anyone': 6405,
 'know': 15884,
 'bulls': 17109,
 'got': 11273,
 'trapped': 805,
 'again': 15121,
 'how': 18312,
 'can': 27571,
 'be': 73598,
 'l': 490,
 'm': 2849,
 'f': 1494,
 'a': 172272,
 'o': 606,
 'by': 24593,
 'friday': 4598,
 'cmon': 986,
 'make': 12668,
 'me': 21107,
 'rich': 2570,
 'daddy': 646,
 'algo': 798,
 'last': 14108,
 'night': 3241,
 'predicts': 122,
 'this': 136160,
 'my': 38410,
 'warning': 749,
 'u': 5117,
 'will': 81946,
 'never': 8239,
 'see': 25858,
 'bull': 9650,
 'run': 11348,
 'until': 7025,
 'fix': 421,
 'oil': 4838,
 'systems': 267,
 'sd': 25,
 'open': 6172,
 'your': 28389,
 'eyes': 983,
 'and': 183010,
 'that': 61585,
 'just': 46931,
 'bouncing': 625,
 'back': 26958,
 'up': 64385,
 'elon': 1895,
 'musk': 701,
 'knows': 2224,
 'total': 1652,
 'economy': 

In [5]:
# K_dict
k_dict = len(word_occurence)
k_dict

114174

In [6]:
# Refine corpus by removing low occurence words and 1 or 2 letter words
word_occurence = {key:val for key, val in word_occurence.items() if val > 15 and len(key) > 2}
word_occurence

{'crypto': 26145,
 'even': 12678,
 'worthless': 684,
 'there': 15661,
 'world': 8206,
 'why': 12106,
 'crashing': 1117,
 'fast': 2855,
 'anyone': 6405,
 'know': 15884,
 'bulls': 17109,
 'got': 11273,
 'trapped': 805,
 'again': 15121,
 'how': 18312,
 'can': 27571,
 'friday': 4598,
 'cmon': 986,
 'make': 12668,
 'rich': 2570,
 'daddy': 646,
 'algo': 798,
 'last': 14108,
 'night': 3241,
 'predicts': 122,
 'this': 136160,
 'warning': 749,
 'will': 81946,
 'never': 8239,
 'see': 25858,
 'bull': 9650,
 'run': 11348,
 'until': 7025,
 'fix': 421,
 'oil': 4838,
 'systems': 267,
 'open': 6172,
 'your': 28389,
 'eyes': 983,
 'and': 183010,
 'that': 61585,
 'just': 46931,
 'bouncing': 625,
 'back': 26958,
 'elon': 1895,
 'musk': 701,
 'knows': 2224,
 'total': 1652,
 'economy': 4542,
 'depends': 188,
 'fuel': 422,
 'lets': 19287,
 'the': 361164,
 'priced': 2299,
 'shit': 9161,
 'hilarious': 504,
 'bears': 29603,
 'right': 14894,
 'now': 41587,
 'say': 6987,
 'hello': 748,
 'holy': 1478,
 'things': 

In [7]:
k_dict = len(word_occurence)
k_dict

13239

In [ ]:
# P(Pos) and P(Neg); N(Pos) and N(Neg)
p_pos = {}
p_neg = {}
n_pos = {}
n_neg = {}

for unique_word in tqdm(word_occurence.keys()):

    rowidx_containword = stocktwit_df["Message"].apply(lambda row: unique_word in row.split())
    df_containword = stocktwit_df[rowidx_containword]

    df_containword_pos = df_containword[df_containword["Sentiment"] == "Bullish"]
    count_in_posclass = df_containword_pos["Message"].apply(lambda row: row.split().count(unique_word)).sum()

    df_containword_neg = df_containword[df_containword["Sentiment"] == "Bearish"]
    count_in_negclass = df_containword_neg["Message"].apply(lambda row: row.split().count(unique_word)).sum()
    
    assert word_occurence[unique_word] == (count_in_posclass+count_in_negclass)
    
    n_pos[unique_word] = count_in_posclass
    n_neg[unique_word] = count_in_negclass
    p_pos[unique_word] = count_in_posclass / word_occurence[unique_word]
    p_neg[unique_word] = count_in_negclass / word_occurence[unique_word]
    
    assert (p_pos[unique_word] + p_neg[unique_word]) == 1
    

In [52]:
# N(Every Pos) and N(Every Neg)
stocktwit_df_pos = stocktwit_df[stocktwit_df["Sentiment"] == "Bullish"]
stocktwit_df_neg = stocktwit_df[stocktwit_df["Sentiment"] == "Bearish"]

n_every_pos = len(stocktwit_df_pos["Message"].str.cat(sep=' ').split())
print(n_every_pos)
n_every_neg = len(stocktwit_df_neg["Message"].str.cat(sep=' ').split())
print(n_every_neg)
n_corpus = len(stocktwit_df["Message"].str.cat(sep=' ').split())
print(n_corpus)
assert n_corpus == (n_every_pos+n_every_neg)

6784438
2799717
9584155


In [53]:
n_every_pos

6784438

In [54]:
# Score_prob(W)
probabilistic_score = {}

for word, freq in word_occurence.items():
    p_word_g_pos = ((n_pos[word] / n_every_pos) + 1) / (k_dict + 1)
    p_word_g_neg = ((n_neg[word] / n_every_neg) + 1) / (k_dict + 1)
    p_pos_g_word = (p_pos[word] *  p_word_g_pos) / freq
    p_neg_g_word = (p_neg[word] *  p_word_g_neg) / freq
    probabilistic_score[word] = p_pos_g_word - p_neg_g_word

In [55]:
probabilistic_score

{'crypto': 2.148120032241357e-09,
 'even': 1.8219054513635102e-09,
 'worthless': 7.10042277614196e-09,
 'there': 1.8921375575695464e-09,
 'world': 3.6617392994043477e-09,
 'why': 2.7071306210153533e-09,
 'crashing': -4.4242694497590335e-09,
 'fast': 9.166396458185093e-09,
 'anyone': 6.458281600102852e-09,
 'know': 2.5364396587651704e-09,
 'bulls': -1.7763292364113776e-09,
 'got': 3.347708048342275e-09,
 'trapped': -3.536886448754294e-10,
 'again': 1.8832330242694147e-09,
 'how': 1.4957423745267892e-09,
 'can': 1.3520593571315139e-09,
 'friday': 3.1433539761198333e-09,
 'cmon': 4.693023556592241e-08,
 'make': 2.8875147094553704e-09,
 'rich': 1.6861227175823712e-08,
 'daddy': 6.154049347318898e-08,
 'algo': 4.1515693437141e-08,
 'last': 1.905750550532661e-09,
 'night': 1.0379506258835222e-08,
 'predicts': 2.3342917484730195e-07,
 'this': 2.35719554683172e-10,
 'warning': -3.407275200466041e-08,
 'will': 3.63825992870436e-10,
 'never': 4.371413367051898e-09,
 'see': 1.2739176084658513e-09

In [56]:
sorted(probabilistic_score, key=probabilistic_score.get, reverse=False)[:50]

['timberrrrrrr',
 'cancellations',
 'bools',
 'shootdown',
 'slivers',
 'gringos',
 'overextension',
 'foreclosures',
 'hopkins',
 'seth',
 'currencywars',
 'mayday',
 'holdforbroke',
 'evacuating',
 'moldova',
 'crapdano',
 'subprime',
 'escalator',
 'economys',
 'dumper',
 'foreigners',
 'taiwans',
 'bombers',
 'singleday',
 'johns',
 'insolvent',
 'gonn',
 'suspends',
 'entourage',
 'trumpinomics',
 'grantham',
 'abort',
 'malaria',
 'failarmy',
 'electing',
 'evacuated',
 'parasite',
 'unitedtraders',
 'stockresearch',
 'coronavirusjhuedumaphtml',
 'constitutional',
 'stockmarkets',
 'fatalities',
 'bcbacker',
 'fraudsters',
 'quicksand',
 'yellin',
 'crashy',
 'marine',
 'mainland']

In [57]:
# Relative Term Frequency of word w in review r
rtf_pos_dict = {key: 0 for key in word_occurence.keys()}
rtf_neg_dict = {key: 0 for key in word_occurence.keys()}

def calculate_rtf(msg, rtf_dict):
    for word in rtf_dict.keys():
        word_list = msg.split(" ")
        if word_list.count(word) != 0:
            rtf_dict[word] += word_list.count(word)/len(word_list)

In [58]:
stocktwit_df_pos["Message"].progress_apply(lambda row: calculate_rtf(row, rtf_pos_dict))
stocktwit_df_neg["Message"].progress_apply(lambda row: calculate_rtf(row, rtf_neg_dict))

100%|██████████████████████████████████████████████████████████████████████████| 197167/197167 [41:16<00:00, 79.63it/s]


0         None
1         None
2         None
3         None
4         None
          ... 
680497    None
680514    None
680527    None
680531    None
680809    None
Name: Cleaned_message, Length: 197167, dtype: object

In [59]:
# Pos(w) and Neg(w)
pos_w = {key: 0 for key in word_occurence.keys()}
neg_w = {key: 0 for key in word_occurence.keys()}

for word in word_occurence.keys():
    pos_w[word] = rtf_pos_dict[word] / len(stocktwit_df_pos) * len(stocktwit_df)
    neg_w[word] = rtf_neg_dict[word] / len(stocktwit_df_neg) * len(stocktwit_df)

In [62]:
# Document Frequency of word w
dfw_dict = {key: 0 for key in word_occurence.keys()}

def calculate_dfw(msg):
    for word in word_occurence.keys():
        word_list = msg.split(" ")
        if word_list.count(word) != 0:
            dfw_dict[word] += 1

stocktwit_df["Message"].progress_apply(lambda row: calculate_dfw(row))
dfw_dict

100%|████████████████████████████████████████████████████████████████████████| 680933/680933 [4:24:03<00:00, 42.98it/s]


{'crypto': 22797,
 'even': 12128,
 'worthless': 669,
 'there': 14587,
 'world': 7616,
 'why': 11572,
 'crashing': 1095,
 'fast': 2749,
 'anyone': 6302,
 'know': 14775,
 'bulls': 16564,
 'got': 10843,
 'trapped': 795,
 'again': 14514,
 'how': 17320,
 'can': 25579,
 'friday': 4446,
 'cmon': 978,
 'make': 12063,
 'rich': 2435,
 'daddy': 635,
 'algo': 726,
 'last': 13573,
 'night': 3095,
 'predicts': 116,
 'this': 117572,
 'warning': 728,
 'will': 67578,
 'never': 7796,
 'see': 24494,
 'bull': 9255,
 'run': 10711,
 'until': 6820,
 'fix': 408,
 'oil': 4252,
 'systems': 249,
 'open': 5995,
 'your': 24242,
 'eyes': 973,
 'and': 130436,
 'that': 52170,
 'just': 43792,
 'bouncing': 622,
 'back': 25602,
 'elon': 1785,
 'musk': 667,
 'knows': 2171,
 'total': 1537,
 'economy': 4180,
 'depends': 187,
 'fuel': 410,
 'lets': 18781,
 'the': 219107,
 'priced': 2163,
 'shit': 8769,
 'hilarious': 502,
 'bears': 28339,
 'right': 14276,
 'now': 39322,
 'say': 6717,
 'hello': 740,
 'holy': 1470,
 'things': 

In [63]:
# Inverse Document Frequency of word w
idf_w = {key: 0 for key in word_occurence.keys()}
info_score = {}

for word in word_occurence.keys():
    idf_w[word] = math.log(len(stocktwit_df) / dfw_dict[word])
    info_score[word] = (pos_w[word] - neg_w[word]) * idf_w[word]
    
info_score

{'crypto': 3768.1020955700237,
 'even': -1137.7048485276987,
 'worthless': -494.9113519191762,
 'there': -344.9417823203607,
 'world': -130.5086235803221,
 'why': 272.063088117398,
 'crashing': -1576.6820309554666,
 'fast': -552.7723259939795,
 'anyone': 721.1707591088663,
 'know': 1206.10223729124,
 'bulls': -14321.179426916684,
 'got': 799.6626062504627,
 'trapped': -643.3240029932114,
 'again': -730.5207806109934,
 'how': -592.746514960124,
 'can': 1024.733133889699,
 'friday': -1235.1438641641337,
 'cmon': 936.0189399716883,
 'make': 554.1947896862671,
 'rich': 508.7672980649163,
 'daddy': 358.76568702262546,
 'algo': -115.92741975653516,
 'last': -642.4197034213956,
 'night': -10.995767350334896,
 'predicts': -2.8280855654005137,
 'this': -24.139282907940842,
 'warning': -834.8141108977594,
 'will': -1076.7289711829844,
 'never': 526.3903774702582,
 'see': 102.92389841999018,
 'bull': -2643.537943669489,
 'run': 2983.062779086736,
 'until': -566.3400907081392,
 'fix': -109.3974445

In [64]:
max_probabilistic_score = max(probabilistic_score.values())
min_probabilistic_score = min(probabilistic_score.values())
max_info_score = max(info_score.values())
min_info_score = min(info_score.values())

mean_probabilistic_score = statistics.mean(probabilistic_score.values())
std_probabilistic_score = statistics.stdev(probabilistic_score.values())
mean_info_score = statistics.mean(info_score.values())
std_info_score = statistics.stdev(info_score.values())

In [85]:
# # std
# hybrid_score = {}

# for word in word_occurence.keys():
#     norm_probabilistic_score = 2*(probabilistic_score[word] - min_probabilistic_score) / (max_probabilistic_score-min_probabilistic_score) -1
#     norm_info_score = 2*(info_score[word] - min_info_score) / (max_info_score-min_info_score) -1
    
#     hybrid_score[word] = (norm_probabilistic_score + norm_info_score) / 2
# hybrid_score

# norm
hybrid_score = {}

for word in word_occurence.keys():
    sta_probabilistic_score = (probabilistic_score[word] - mean_probabilistic_score) / std_probabilistic_score
    sta_info_score = (info_score[word] - mean_info_score) / std_info_score
    hybrid_score[word] = (sta_probabilistic_score + sta_info_score) / 2
    
hybrid_score

{'crypto': 4.332423751519165,
 'even': -1.6791295550727128,
 'worthless': -0.8890238024882591,
 'there': -0.7076732050088534,
 'world': -0.4440932661623169,
 'why': 0.04876087298341869,
 'crashing': -2.2199350424296678,
 'fast': -0.9589659167240272,
 'anyone': 0.6008221376314156,
 'know': 1.1932201319375881,
 'bulls': -17.83536167606756,
 'got': 0.6955598674353738,
 'trapped': -1.074342330731982,
 'again': -1.1801521738834904,
 'how': -1.0115082794608439,
 'can': 0.9704272744010559,
 'friday': -1.7979146800338999,
 'cmon': 0.8828696567771199,
 'make': 0.39455876288980746,
 'rich': 0.345377737474809,
 'daddy': 0.18230357078656712,
 'algo': -0.40866063055727686,
 'last': -1.072185788016088,
 'night': -0.2945292075446365,
 'predicts': -0.18101922963531622,
 'this': -0.31534182024581386,
 'warning': -1.3246341558858843,
 'will': -1.6050884680293471,
 'never': 0.3611767942541225,
 'see': -0.15916134749132432,
 'bull': -3.5244055199715456,
 'run': 3.3716003469277833,
 'until': -0.97837204970

In [86]:
hybrid_score_df = pd.DataFrame.from_dict(hybrid_score, orient="index")
hybrid_score_df

,0
crypto,4.332424
even,-1.679130
worthless,-0.889024
there,-0.707673
world,-0.444093
...,...
goodvibes,2.085768
solflare,1.581892
soldana,-1.878540
solsea,0.843677


In [87]:
hybrid_score_df.to_csv("domain_lexicon_norm.csv", header=False)